# OZ01:Explore hologram data  quality on Hologram For Ozone

- author Sylvie Dagoret-Campagne
- creation date 2025-09-16 : version v1
- affiliation : IJCLab
- Kernel @usdf **w_2025_36*
- Home emac : base (conda)
- laptop : conda_py310

**Goal** : Show Night variations of Ozone wrt date and Time. Fit a straight line.

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_Ozone01"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove this if want to run faster the ntoebook
# import ipywidgets as widgets
# %matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

## Configuration

In [ ]:
version_results = "v1"
legendtag = {"v1" : "v3.1.0"}

In [ ]:
atmfilenamesdict = {"v1":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_atmosphere_20250912a_repomain_v1.npy",}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

#### Series on spec

In [ ]:
ser_spec_size = df_spec.groupby(["nightObs"]).size()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(ser_spec_size.values,bins=50,facecolor="b")
ax.set_title("nb obs per night")
ax.set_xlabel("Nobs/night")

##### Make 3 series

In [ ]:
ser_CHI2_FIT = df_spec[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_O3 = df_spec[["ozone [db]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_O3_CHI2_FIT = df_spec[["ozone [db]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
#ser_O3

In [ ]:
#ser_CHI2_FIT

In [ ]:
#ser_O3_CHI2_FIT 

### Plot Ozone and Chi2 before anly selection

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean Ozone per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median Ozone per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for Ozone per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="r",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night")
plt.tight_layout()

In [ ]:
#ser_O3_CHI2_FIT 

In [ ]:
ser_O3_CHI2_FIT.loc[20230117,:]

In [ ]:
df_unstack = ser_O3_CHI2_FIT.loc[20230117,:].unstack()
df_unstack

In [ ]:
ser_O3_CHI2_FIT.loc[20230117,"ozone [db]_x"]["count"]

In [ ]:
df_unstack.loc["ozone [db]_x","count"]

#### Add aggregate data added to pandas dataframe

In [ ]:
def FillAgreggates(row):
    the_nightObs = row["nightObs"]
    df_night = ser_O3_CHI2_FIT.loc[the_nightObs,:].unstack()
    count = df_night.loc["ozone [db]_x","count"]
    o3min = df_night.loc["ozone [db]_x","min"]
    o3max = df_night.loc["ozone [db]_x","max"]
    o3mean = df_night.loc["ozone [db]_x","mean"]
    o3median = df_night.loc["ozone [db]_x","median"]
    o3std = df_night.loc["ozone [db]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_o3min":o3min,"_o3max":o3max,"_o3mean":o3mean,"_o3median":o3median,"_o3std":o3std,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
row = df_spec.iloc[0]

In [ ]:
row

In [ ]:
FillAgreggates(row)

#### join dataframe + aggregates

In [ ]:
df_spec = df_spec.join(df_spec.apply(FillAgreggates,axis=1,result_type="expand"))

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

### Add the Time in pd.datetime

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()
df_spec.plot(x="Time",y="ozone [db]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("ozone [db]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Ozone meas by holo vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

## Apply Quality selection cuts

In [ ]:
def getSelectionCutOld(df_spec, chi2max=20., o3min=1.0, o3max = 599.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["ozone [db]_x"] > o3min) & (df_spec["ozone [db]_x"] < o3max) 
    return cut

In [ ]:
#def getSelectionCut(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
def getSelectionCut(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["ozone [db]_x"] > ozmin) & (df_spec["ozone [db]_x"] < ozmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0)
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
cut = getSelectionCut(df_spec)

In [ ]:
df_spec_sel = df_spec[cut].drop(labels=['_count', '_o3min', '_o3max', '_o3mean', '_o3median', '_o3std','_chi2min', '_chi2max', '_chi2mean', '_chi2median', '_chi2std'],axis=1)

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)

## Compute per-night aggregates

### Compute series per night

In [ ]:
ser_O3_CHI2_FIT_sel = df_spec_sel[["ozone [db]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
def FillAgreggatesSel(row):
    the_nightObs = row["nightObs"]
    df_night = ser_O3_CHI2_FIT_sel.loc[the_nightObs,:].unstack()
    count = df_night.loc["ozone [db]_x","count"]
    o3min = df_night.loc["ozone [db]_x","min"]
    o3max = df_night.loc["ozone [db]_x","max"]
    o3mean = df_night.loc["ozone [db]_x","mean"]
    o3median = df_night.loc["ozone [db]_x","median"]
    o3std = df_night.loc["ozone [db]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_o3min":o3min,"_o3max":o3max,"_o3mean":o3mean,"_o3median":o3median,"_o3std":o3std,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
df_spec_sel = df_spec_sel.join(df_spec_sel.apply(FillAgreggatesSel,axis=1,result_type="expand"))

## Recompute night boundaries

In [ ]:
dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

df_spec_sel.plot(x="Time",y="ozone [db]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg,ms=10)
ax.set_ylabel("ozone [db]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Ozone measured by holo selected vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()
figname =f"{pathfigs}/ozone_allpoints_allnights"+figtype
plt.savefig(figname)
plt.show()

## Plot series on selected data

In [ ]:
ser_CHI2_FIT_sel = df_spec_sel[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_O3_sel = df_spec_sel[["ozone [db]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean Ozone per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median Ozone per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for Ozone per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night after selection")
plt.tight_layout()

## Show time variation each night

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_pwv_curve):
    """
    """

    x = df_night_pwv_curve["dt"].values
    y = df_night_pwv_curve["ozone [db]_x"].values
    yerr = df_night_pwv_curve["ozone [db]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

### Plot night by night

In [ ]:
all_dateObs_sel = {}

# loop on nights
for night in all_selected_nights:
    #select the night
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    
    #select the variables
    df_night_o3_curve = df_spec_night[["Time","ozone [db]_x","ozone [db]_err_x"]]

    tmin = df_night_o3_curve["Time"].min()

    # convert in hours
    df_night_o3_curve["dt"] = (df_night_o3_curve["Time"] - tmin).dt.total_seconds()/3600.
    
    
    stat = df_night_o3_curve[["ozone [db]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))


    

    N= len(df_night_o3_curve)

    if N>10:
        # does the fit
        
        x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err = MakeLineFit(df_night_o3_curve)
        
        textstr2 = "\n".join((f"d(Ozone/dt) : ",
                     f"slope : {slope:.3f} DU/h ",
                     f"slope err : {slope_err:.3f} DU/h",
                     f"chi2/ndeg : {chi2dof:.2f}",       
                    ))

        textstr3 = "\n".join((f"d(Ozone/dt) : ",
                     f"slope : {slope:.3f} DU/h ",
                     f"slope err : {slope_err:.3f} DU/h",  
                    ))
    

        stat.loc["slope","ozone [db]_x"] = slope 
        stat.loc["slope_err","ozone [db]_err_x"] = slope_err 
        stat.loc["chi2","ozone [db]_x"] = chi2dof
        
        # plot
        
        fig = plt.figure(figsize=(16,6))
        gs = GridSpec(1, 2,  width_ratios=[2,1],figure=fig)

        # left figure
        ax = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1],sharey=ax)
        #ax2 = fig.add_subplot(gs[1])
        
        df_night_o3_curve.plot(x="Time",y="ozone [db]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,legend=leg,label=tag,ms=10)
        ax.errorbar(x=df_night_o3_curve.Time, y=df_night_o3_curve["ozone [db]_x"],yerr=df_night_o3_curve["ozone [db]_err_x"],fmt=".",color="r",ecolor="k")
        
        ax.legend(loc="upper right")
        ax.set_ylabel("ozone [db]_x")
        ax.set_xlabel("time")
        ax.xaxis.set_major_formatter(date_form)
        ax.set_ylim(0.,600.)
        ax.text(0.03, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
        ax.text(0.35, 0.95, textstr3, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
        ax.set_title(f"night {night}")
        
        # right figure
        ax2.errorbar(x=df_night_o3_curve.dt, y=df_night_o3_curve["ozone [db]_x"],yerr=df_night_o3_curve["ozone [db]_err_x"],fmt="+",color="r",ecolor="k",ms=10,label=tag)
        ax2.set_xlabel("hours")
        ax2.set_ylim(0.,600.)
        ax2.grid()
        ax2.legend(loc="upper right")
        ax2.plot(xfit,yfit,"k-")
        ax2.text(0.45, 0.95, textstr2, transform=ax2.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
        ax2.set_title(f"night {night}")
        
        plt.tight_layout()
        figname =f"{pathfigs}/o3_per_night_{night}"+figtype
        plt.savefig(figname)
        plt.show()

        # add statistics
        all_dateObs_sel[night] = stat

## Make a summary of Night quality

In [ ]:
def GetStatistics(all_dateObs_sel):
    df = pd.DataFrame(columns = ["count","mean","median","std","slope","slope_err"])
    
    for nightObs, stat in all_dateObs_sel.items():
        count = int(stat.loc["count"].values[0])
        mean = stat.loc["mean"].values[0]
        median = stat.loc["50%"].values[0]
        std = stat.loc["std"].values[0]
        slope = stat.loc["slope"].values[0]
        slope_err = stat.loc["slope_err"].values[0]
        df.loc[nightObs] = [count,mean,median,std,slope,slope_err]
    return df

In [ ]:
df = GetStatistics(all_dateObs_sel)

### Convert dateobs into datetime

In [ ]:
def ComputeDataTimeStr(num):
    year = num//10000
    remain = (num-year*10000)
    month = remain//100
    day = remain-100*month
    yearstr = f"{year}"
    monthstr = f"{month}"
    daystr= f"{day}"
    stry = yearstr.zfill(4)
    strm = monthstr.zfill(2)
    strd = daystr.zfill(2)
    
    str = f"{stry}-{strm}-{strd}"
    return str

In [ ]:
df["Time"] = pd.to_datetime([ ComputeDataTimeStr(num) for  num in df.index])

### Plot summary

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(2,1,figsize=(12,8),sharex=True)
ax1,ax2  = axs

leg1=ax1.get_legend()
leg2=ax2.get_legend()


df.plot(x="Time",y="median",ax=ax1,marker='+',c="r",lw=0.0,grid=True,ms=10,label=tag,legend=leg1)
ax1.errorbar(x=df["Time"], y=df["median"],yerr=df["std"],fmt=".",color="r",ecolor="k")

ax1.set_ylabel("Ozone (DU)")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Median Ozone per night")
ax1.set_ylim(0.,600.)
ax1.legend(loc="upper right")


ax2.set_title("Fitted drift per night")
ax2.errorbar(x=df["Time"], y=df["slope"],yerr=df["slope_err"],fmt=".",color="r",ecolor="k",ms=10,label=tag,legend=leg2)
ax2.grid()
ax2.set_ylabel("dOzone/dt (DU/h)")
ax2.set_xlabel("time")
ax2.set_ylim(-50.,50.)
ax2.xaxis.set_major_formatter(date_form)
ax2.legend(loc="upper right")

ax2.tick_params(axis='x', labelrotation=45)


plt.tight_layout()
figname =f"{pathfigs}/ozone_medianspredslope_allnights"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax = axs
leg=ax.get_legend()


df.plot(x="Time",y="median",ax=ax,marker='+',c="r",ms=20,lw=0.0,grid=True,label=tag,legend=leg)
ax.errorbar(x=df["Time"], y=df["median"],yerr=df["std"],fmt="o",ms=5,color="r",ecolor="k")
ax.set_ylabel("Ozone (DU)")
ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Median and spread of Ozone per night")
ax.set_ylim(0.,600)

ax.legend(loc="upper right")

figname =f"{pathfigs}/ozone_medians_allnights"+figtype
plt.savefig(figname)
plt.show()
